In [1]:
from setup import *

In [20]:
import pandas as pd
import numpy as np
import cv2
from read_roi import read_roi_file
import os
import random

# Define the functions here (assumed to be already provided)

def detect_features_in_channel(channel):
    features = cv2.goodFeaturesToTrack(channel, maxCorners=200, qualityLevel=0.01, minDistance=50, blockSize=300)
    if features is not None:
        return np.int0(features)
    else:
        return np.array([])

def filter_close_points(features, min_dist=100):
    if features.ndim == 3:
        features = features.reshape(-1, 2)
    
    filtered_points = []

    for feature in features:
        if not filtered_points:
            filtered_points.append(feature)
            continue

        dists = np.sqrt(np.sum((np.array(filtered_points) - feature) ** 2, axis=1))

        if np.all(dists >= min_dist):
            filtered_points.append(feature)

    return np.array(filtered_points)

def bounding_box(roi_file_path):
    roi = read_roi_file(roi_file_path)
    for box_info in roi.values():
        if box_info['type'] == 'rectangle':
            left = box_info['left']
            top = box_info['top']
            width = box_info['width']
            height = box_info['height']
    print(left, top, width, height)
    return left, top, width, height

def extract_patch(image, center_x, center_y, patch_size):
    half_width, half_height = patch_size[0] // 2, patch_size[1] // 2
    patch = image[
        center_y - half_height:center_y + half_height,
        center_x - half_width:center_x + half_width
    ]
    return patch

def extract_patches_from_abnormal(image_path, roi_file_path, patch_size=(100, 100)):
    image = cv2.imread(image_path)
    print(f'Image:{image}')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    left, top, width, height = bounding_box(roi_file_path)
    roi_region = gray_image[top:top+height, left:left+width]
    
    features = detect_features_in_channel(roi_region)
    
    if len(features) > 0:
        features[:, 0] += left
        features[:, 1] += top

        salient_points = filter_close_points(features)
        
        if len(salient_points) > 0:
            center_x, center_y = np.mean(salient_points, axis=0).astype(int)
            patch = extract_patch(image, center_x, center_y, patch_size)
            visualize_patch(image, center_x, center_y, patch_size)
            return patch, image
    
    center_x, center_y = left + width // 2, top + height // 2
    patch = extract_patch(image, center_x, center_y, patch_size)
    visualize_patch(image, center_x, center_y, patch_size)
    
    return patch, image

def extract_patches_from_normal(image_path, patch_size=(100, 100)):
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    
    middle_x_range = (w // 4, 3 * w // 4)
    middle_y_range = (h // 4, 3 * h // 4)
    
    patches = []
    visualized_images = []
    for _ in range(2):
        center_x = random.randint(middle_x_range[0], middle_x_range[1])
        center_y = random.randint(middle_y_range[0], middle_y_range[1])
        patch = extract_patch(image, center_x, center_y, patch_size)
        patches.append(patch)
        
        visualize_patch(image, center_x, center_y, patch_size)
        visualized_images.append(image.copy())
    
    return patches, visualized_images

def visualize_patch(image, center_x, center_y, patch_size):
    half_width, half_height = patch_size[0] // 2, patch_size[1] // 2
    top_left = (center_x - half_width, center_y - half_height)
    bottom_right = (center_x + half_width, center_y + half_height)
    cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)

def save_patches(patches, output_dir, base_name):
    for i, patch in enumerate(patches):
        patch_filename = f"{base_name}_patch_{i}.png"
        patch_path = os.path.join(output_dir, patch_filename)
        cv2.imwrite(patch_path, patch)

def save_visualized_images(images, output_dir, base_name):
    for i, img in enumerate(images):
        visualized_filename = f"{base_name}_visualized_{i}.png"
        visualized_path = os.path.join(output_dir, visualized_filename)
        cv2.imwrite(visualized_path, img)

def process_directory(input_dir, output_dir, visualized_dir, is_abnormal=False, patch_size=(100, 100)):
    print(f"Processing directory: {input_dir}")
    for filename in os.listdir(input_dir):
        print(filename)
        if filename.endswith('.png'):
            image_path = os.path.join(input_dir, filename)
            base_name = os.path.splitext(filename)[0]

            if is_abnormal:
                roi_file_path = os.path.join(input_dir, f"{base_name}.roi")
                patch, visualized_image = extract_patches_from_abnormal(image_path, roi_file_path, patch_size)
                save_patches([patch], output_dir, base_name)
                save_visualized_images([visualized_image], visualized_dir, base_name)
            else:
                patches, visualized_images = extract_patches_from_normal(image_path, patch_size)
                save_patches(patches, output_dir, base_name)
                save_visualized_images(visualized_images, visualized_dir, base_name)

# Define train and test directories
train_dir = '../Data/Split_Data/train'
test_dir = '../Data/Split_Data/test'

# Process the normal and abnormal directories in both train and test directories
for data_dir in [train_dir, test_dir]:
    for class_type in ['normal', 'abnormal']:
        input_dir = os.path.join(data_dir, class_type)
        print(input_dir)
        output_dir = os.path.join('Processed_Data', os.path.basename(data_dir), class_type)
        visualized_dir = os.path.join('Visualized_Data', os.path.basename(data_dir), class_type)
        os.makedirs(output_dir, exist_ok=True)
        os.makedirs(visualized_dir, exist_ok=True)
        process_directory(input_dir, output_dir, visualized_dir, is_abnormal=(class_type == 'abnormal'))

print("Patch extraction and visualization complete.")


../Data/Split_Data/train/normal
Processing directory: ../Data/Split_Data/train/normal
356-1-61-adenoma-D-0001.roi
037-1-74-adenoma-T-0002.jpg
102-1-76-polyp-S-0008.jpg
125-1-68-polyp-T-0005.roi
284-2-77-adenoma-HF-0001.roi
027-1-79-adenoma-T-0000.roi
043-1-53-adenoma-T-0001.jpg
513-1-59-adenoma-D-0001.roi
402-1-74-adenoma-D-0005.roi
287-1-60-adenoma-A-0001.roi
136-1-50-adenoma-C-0000.roi
429-1-54-adenoma-S-0004.jpg
146-2-62-adenoma-A-0000.jpg
311-1-58-adenoma-S-0001.jpg
527-1-54-adenoma-S-0001.roi
332-1-62-adenoma-HF,T-0002.jpg
188-2-78-adenoma-35cm-0000.jpg
097-1-76-adenoma-T-0001.jpg
292-1-65-adenoma-C-0001.roi
342-1-58-polyp-R-0001.jpg
496-2-84-adenoma-A-0002.jpg
314-1-72-adenoma-S-0003.jpg
537-1-63-adenoma-C-0002.roi
331-1-62-adenoma-A-0002.jpg
538-2-66-2-adenoma-T-0001.jpg
046-1-76-adenoma-R-0009.roi
167-1-54-adenoma-D-0000.roi
035-1-56-adenoma-R-0002.roi
127-1-63-adenoma-T-0001.jpg
122-2-60-adenoma-R-0000.roi
404-1-74-adenoma-S-0007.jpg
461-1-70-adenoma-D-0007.roi
142-2-64-adenom

In [6]:
import pandas as pd
import numpy as np
import cv2
from read_roi import read_roi_file
import os
import random

def detect_features_in_channel(channel):
    features = cv2.goodFeaturesToTrack(channel, maxCorners=200, qualityLevel=0.01, minDistance=50, blockSize=300)
    if features is not None:
        return np.int0(features)
    else:
        return np.array([])

def filter_close_points(features, min_dist=100):
    if features.ndim == 3:
        features = features.reshape(-1, 2)
    
    filtered_points = []

    for feature in features:
        if not filtered_points:
            filtered_points.append(feature)
            continue

        dists = np.sqrt(np.sum((np.array(filtered_points) - feature) ** 2, axis=1))

        if np.all(dists >= min_dist):
            filtered_points.append(feature)

    return np.array(filtered_points)

def bounding_box(roi_file_path):
    roi = read_roi_file(roi_file_path)
    for box_info in roi.values():
        if box_info['type'] == 'rectangle':
            left = box_info['left']
            top = box_info['top']
            width = box_info['width']
            height = box_info['height']
    return left, top, width, height

def extract_patch(image, center_x, center_y, patch_size):
    half_width, half_height = patch_size[0] // 2, patch_size[1] // 2
    patch = image[
        center_y - half_height:center_y + half_height,
        center_x - half_width:center_x + half_width
    ]
    return patch

def extract_patches_from_abnormal(image_path, roi_file_path, patch_size=(100, 100)):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    left, top, width, height = bounding_box(roi_file_path)
    roi_region = gray_image[top:top+height, left:left+width]
    
    features = detect_features_in_channel(roi_region)
    
    if len(features) > 0:
        features[:, 0] += left
        features[:, 1] += top

        salient_points = filter_close_points(features)
        
        if len(salient_points) > 0:
            center_x, center_y = np.mean(salient_points, axis=0).astype(int)
            patch = extract_patch(image, center_x, center_y, patch_size)
            visualize_patch(image, center_x, center_y, patch_size)
            return patch, image
    
    center_x, center_y = left + width // 2, top + height // 2
    patch = extract_patch(image, center_x, center_y, patch_size)
    visualize_patch(image, center_x, center_y, patch_size)
    
    return patch, image

def extract_patches_from_normal(image_path, patch_size=(100, 100)):
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    
    middle_x_range = (w // 4, 3 * w // 4)
    middle_y_range = (h // 4, 3 * h // 4)
    
    patches = []
    visualized_images = []
    for _ in range(2):
        center_x = random.randint(middle_x_range[0], middle_x_range[1])
        center_y = random.randint(middle_y_range[0], middle_y_range[1])
        patch = extract_patch(image, center_x, center_y, patch_size)
        patches.append(patch)
        
        visualize_patch(image, center_x, center_y, patch_size)
        visualized_images.append(image.copy())
    
    return patches, visualized_images

def visualize_patch(image, center_x, center_y, patch_size):
    half_width, half_height = patch_size[0] // 2, patch_size[1] // 2
    top_left = (center_x - half_width, center_y - half_height)
    bottom_right = (center_x + half_width, center_y + half_height)
    cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)

def save_patches(patches, output_dir, base_name):
    for i, patch in enumerate(patches):
        patch_filename = f"{base_name}_patch_{i}.png"
        patch_path = os.path.join(output_dir, patch_filename)
        cv2.imwrite(patch_path, patch)

def save_visualized_images(images, output_dir, base_name):
    for i, img in enumerate(images):
        visualized_filename = f"{base_name}_visualized_{i}.png"
        visualized_path = os.path.join(output_dir, visualized_filename)
        cv2.imwrite(visualized_path, img)

def process_directory(input_dir, output_dir, visualized_dir, is_abnormal=False, patch_size=(100, 100)):
    print(f"Processing directory: {input_dir}")
    for filename in os.listdir(input_dir):
        if filename.endswith('.png'):
            image_path = os.path.join(input_dir, filename)
            print(f'ImagePath:{image_path}')
            base_name = os.path.splitext(filename)[0]
            print(f'BaseName:{base_name}')
            
            if is_abnormal:
                roi_file_path = os.path.join(input_dir, f"{base_name}.roi")
                print(f'RoiFilePath:{roi_file_path}')
                if os.path.exists(roi_file_path):
                    patch, visualized_image = extract_patches_from_abnormal(image_path, roi_file_path, patch_size)
                    print(f'Patch:{patch}')
                    save_patches([patch], output_dir, base_name)
                    save_visualized_images([visualized_image], visualized_dir, base_name)
            else:
                patches, visualized_images = extract_patches_from_normal(image_path, patch_size)
                print(f'Patches:{patches}')
                save_patches(patches, output_dir, base_name)
                save_visualized_images(visualized_images, visualized_dir, base_name)

# Define train and test directories
train_dir = 'Data/Split_Data/train'
test_dir = 'Data/Split_Data/test'

# Process the normal and abnormal directories in both train and test directories
for data_dir in [train_dir, test_dir]:
    for class_type in ['normal', 'abnormal']:
        input_dir = os.path.join(data_dir, class_type)
        print(input_dir)
        output_dir = os.path.join('Processed_Data', os.path.basename(data_dir), class_type)
        visualized_dir = os.path.join('Visualized_Data', os.path.basename(data_dir), class_type)
        os.makedirs(output_dir, exist_ok=True)
        os.makedirs(visualized_dir, exist_ok=True)
        process_directory(input_dir, output_dir, visualized_dir, is_abnormal=(class_type == 'abnormal'))

print("Patch extraction and visualization complete.")


Data/Split_Data/train/normal
Processing directory: Data/Split_Data/train/normal
Data/Split_Data/train/abnormal
Processing directory: Data/Split_Data/train/abnormal
Data/Split_Data/test/normal
Processing directory: Data/Split_Data/test/normal
Data/Split_Data/test/abnormal
Processing directory: Data/Split_Data/test/abnormal
Patch extraction and visualization complete.


In [10]:
import pandas as pd
import numpy as np
import cv2
from read_roi import read_roi_file
import os
import random

def detect_features_in_channel(channel):
    features = cv2.goodFeaturesToTrack(channel, maxCorners=200, qualityLevel=0.01, minDistance=50, blockSize=300)
    if features is not None:
        return np.int0(features)
    else:
        return np.array([])

def filter_close_points(features, min_dist=100):
    if features.ndim == 3:
        features = features.reshape(-1, 2)
    
    filtered_points = []

    for feature in features:
        if not filtered_points:
            filtered_points.append(feature)
            continue

        dists = np.sqrt(np.sum((np.array(filtered_points) - feature) ** 2, axis=1))

        if np.all(dists >= min_dist):
            filtered_points.append(feature)

    return np.array(filtered_points)

def bounding_box(roi_file_path):
    roi = read_roi_file(roi_file_path)
    for box_info in roi.values():
        if box_info['type'] == 'rectangle':
            left = box_info['left']
            top = box_info['top']
            width = box_info['width']
            height = box_info['height']
    return left, top, width, height

def extract_patch(image, center_x, center_y, patch_size):
    half_width, half_height = patch_size[0] // 2, patch_size[1] // 2
    patch = image[
        center_y - half_height:center_y + half_height,
        center_x - half_width:center_x + half_width
    ]
    return patch

def extract_patches_from_abnormal(image_path, roi_file_path, patch_size=(100, 100)):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    left, top, width, height = bounding_box(roi_file_path)
    roi_region = gray_image[top:top+height, left:left+width]
    
    features = detect_features_in_channel(roi_region)
    
    if len(features) > 0:
        features[:, 0] += left
        features[:, 1] += top

        salient_points = filter_close_points(features)
        
        if len(salient_points) > 0:
            center_x, center_y = np.mean(salient_points, axis=0).astype(int)
            patch = extract_patch(image, center_x, center_y, patch_size)
            visualize_patch(image, center_x, center_y, patch_size)
            return patch, image
    
    center_x, center_y = left + width // 2, top + height // 2
    patch = extract_patch(image, center_x, center_y, patch_size)
    visualize_patch(image, center_x, center_y, patch_size)
    
    return patch, image

def extract_patches_from_normal(image_path, patch_size=(100, 100)):
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    
    middle_x_range = (w // 4, 3 * w // 4)
    middle_y_range = (h // 4, 3 * h // 4)
    
    patches = []
    visualized_images = []
    for _ in range(2):
        center_x = random.randint(middle_x_range[0], middle_x_range[1])
        center_y = random.randint(middle_y_range[0], middle_y_range[1])
        patch = extract_patch(image, center_x, center_y, patch_size)
        patches.append(patch)
        
        visualize_patch(image, center_x, center_y, patch_size)
        visualized_images.append(image.copy())
    
    return patches, visualized_images

def visualize_patch(image, center_x, center_y, patch_size):
    half_width, half_height = patch_size[0] // 2, patch_size[1] // 2
    top_left = (center_x - half_width, center_y - half_height)
    bottom_right = (center_x + half_width, center_y + half_height)
    cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)

def save_patches(patches, output_dir, base_name):
    for i, patch in enumerate(patches):
        patch_filename = f"{base_name}_patch_{i}.png"
        patch_path = os.path.join(output_dir, patch_filename)
        cv2.imwrite(patch_path, patch)

def save_visualized_images(images, output_dir, base_name):
    for i, img in enumerate(images):
        visualized_filename = f"{base_name}_visualized_{i}.png"
        visualized_path = os.path.join(output_dir, visualized_filename)
        cv2.imwrite(visualized_path, img)

def process_abnormal_directory(input_dir, output_dir, visualized_dir, patch_size=(100, 100)):
    print(f"Processing abnormal directory: {input_dir}")
    if not os.path.exists(input_dir):
        print(f"Directory does not exist: {input_dir}")
        return

    for filename in os.listdir(input_dir):
        if filename.endswith('.png'):
            image_path = os.path.join(input_dir, filename)
            base_name = os.path.splitext(filename)[0]
            roi_file_path = os.path.join(input_dir, f"{base_name}.roi")
            if os.path.exists(roi_file_path):
                patch, visualized_image = extract_patches_from_abnormal(image_path, roi_file_path, patch_size)
                save_patches([patch], output_dir, base_name)
                save_visualized_images([visualized_image], visualized_dir, base_name)
            else:
                print(f"ROI file does not exist: {roi_file_path}")

def process_normal_directory(input_dir, output_dir, visualized_dir, patch_size=(100, 100)):
    print(f"Processing normal directory: {input_dir}")
    if not os.path.exists(input_dir):
        print(f"Directory does not exist: {input_dir}")
        return

    for filename in os.listdir(input_dir):
        if filename.endswith('.png'):
            image_path = os.path.join(input_dir, filename)
            base_name = os.path.splitext(filename)[0]
            patches, visualized_images = extract_patches_from_normal(image_path, patch_size)
            save_patches(patches, output_dir, base_name)
            save_visualized_images(visualized_images, output_dir, base_name)

# Define train and test directories
train_dir = 'Data/Split_Data/train'
test_dir = 'Data/Split_Data/test'

# Process the normal and abnormal directories in both train and test directories
for data_dir in [train_dir, test_dir]:
    for class_type in ['normal', 'abnormal']:
        input_dir = os.path.join(data_dir, class_type)
        output_dir = os.path.join('Processed_Data', os.path.basename(data_dir), class_type)
        visualized_dir = os.path.join('Visualized_Data', os.path.basename(data_dir), class_type)
        os.makedirs(output_dir, exist_ok=True)
        os.makedirs(visualized_dir, exist_ok=True)
        if class_type == 'abnormal':
            process_abnormal_directory(input_dir, output_dir, visualized_dir)
        else:
            process_normal_directory(input_dir, output_dir, visualized_dir)

print("Patch extraction and visualization complete.")


Processing normal directory: Data/Split_Data/train/normal
Processing abnormal directory: Data/Split_Data/train/abnormal
Processing normal directory: Data/Split_Data/test/normal
Processing abnormal directory: Data/Split_Data/test/abnormal
Patch extraction and visualization complete.


In [11]:
import os
import cv2
import numpy as np
from read_roi import read_roi_file
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

def detect_features_in_channel(channel):
    features = cv2.goodFeaturesToTrack(channel, maxCorners=200, qualityLevel=0.01, minDistance=50, blockSize=300)
    if features is not None:
        return np.int0(features)
    else:
        return np.array([])

def filter_close_points(features, min_dist=100):
    if features.ndim == 3:
        features = features.reshape(-1, 2)
    
    filtered_points = []

    for feature in features:
        if not filtered_points:
            filtered_points.append(feature)
            continue

        dists = np.sqrt(np.sum((np.array(filtered_points) - feature) ** 2, axis=1))

        if np.all(dists >= min_dist):
            filtered_points.append(feature)

    return np.array(filtered_points)

def bounding_box(roi_file_path):
    roi = read_roi_file(roi_file_path)
    for box_info in roi.values():
        if box_info['type'] == 'rectangle':
            left = box_info['left']
            top = box_info['top']
            width = box_info['width']
            height = box_info['height']
    return left, top, width, height

def extract_and_save_patches_abnormal(folder, output_folder, ideal_patch_size=(275, 300)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    for filename in os.listdir(folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(folder, filename)
            image = cv2.imread(image_path)
            roi_path = os.path.splitext(image_path)[0] + ".roi"

            if os.path.exists(roi_path):
                blue, green, red = cv2.split(image)
                features_blue = detect_features_in_channel(blue)
                features_green = detect_features_in_channel(green)
                features_red = detect_features_in_channel(red)
                features_combined = np.vstack([f.reshape(-1, 2) for f in [features_blue, features_green, features_red] if f.size > 0])
                features_combined = np.unique(features_combined, axis=0)
                features_combined = filter_close_points(features_combined)

                if features_combined.size == 0:
                    continue  # Skip if no features found

                for point in features_combined:
                    cv2.circle(image, (int(point[0]), int(point[1])), radius=5, color=(255, 0, 0), thickness=-1)

                gt_left, gt_top, gt_width, gt_height = bounding_box(roi_path)
                cv2.rectangle(image, (gt_left, gt_top), (gt_left + gt_width, gt_top + gt_height), (0, 255, 0), 3)

                gt_center = np.array([[gt_left + gt_width / 2, gt_top + gt_height / 2]])
                distances = cdist(features_combined, gt_center, metric='euclidean')
                closest_point_index = np.argmin(distances)
                closest_point = features_combined[closest_point_index]

                sp_start_x = max(int(closest_point[0] - ideal_patch_size[1] / 2), 0)
                sp_start_y = max(int(closest_point[1] - ideal_patch_size[0] / 2), 0)
                sp_end_x = min(sp_start_x + ideal_patch_size[1], image.shape[1])
                sp_end_y = min(sp_start_y + ideal_patch_size[0], image.shape[0])

                cv2.rectangle(image, (sp_start_x, sp_start_y), (sp_end_x, sp_end_y), (0, 0, 0), 3)

                salient_patch = image[sp_start_y:sp_end_y, sp_start_x:sp_end_x]
                salient_patch_name = f"{os.path.splitext(os.path.basename(image_path))[0]}_salient_patch_{sp_start_x}_{sp_start_y}.png"
                salient_patch_path = os.path.join(output_folder, salient_patch_name)

                cv2.imwrite(salient_patch_path, salient_patch)

def extract_and_save_patches_normal(folder, output_folder, ideal_patch_size=(275, 300)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(folder, filename)
            image = cv2.imread(image_path)
            h, w, _ = image.shape

            middle_x_range = (w // 4, 3 * w // 4)
            middle_y_range = (h // 4, 3 * h // 4)

            patches = []
            for _ in range(2):
                center_x = random.randint(middle_x_range[0], middle_x_range[1])
                center_y = random.randint(middle_y_range[0], middle_y_range[1])
                sp_start_x = max(int(center_x - ideal_patch_size[1] / 2), 0)
                sp_start_y = max(int(center_y - ideal_patch_size[0] / 2), 0)
                sp_end_x = min(sp_start_x + ideal_patch_size[1], w)
                sp_end_y = min(sp_start_y + ideal_patch_size[0], h)

                patch = image[sp_start_y:sp_end_y, sp_start_x:sp_end_x]
                patches.append(patch)
                patch_name = f"{os.path.splitext(os.path.basename(image_path))[0]}_normal_patch_{sp_start_x}_{sp_start_y}.png"
                patch_path = os.path.join(output_folder, patch_name)
                cv2.imwrite(patch_path, patch)

### Main Function to Process Both Normal and Abnormal Images:

def process_directories(train_dir, test_dir, output_dir, ideal_patch_size=(275, 300)):
    for data_dir in [train_dir, test_dir]:
        for class_type in ['normal', 'abnormal']:
            input_dir = os.path.join(data_dir, class_type)
            output_sub_dir = os.path.join(output_dir, os.path.basename(data_dir), class_type)
            os.makedirs(output_sub_dir, exist_ok=True)

            if class_type == 'abnormal':
                extract_and_save_patches_abnormal(input_dir, output_sub_dir, ideal_patch_size)
            else:
                extract_and_save_patches_normal(input_dir, output_sub_dir, ideal_patch_size)

### Define Train and Test Directories:

train_dir = 'Data/Split_Data/train'
test_dir = 'Data/Split_Data/test'
output_dir = 'Processed_Patches'

### Process the Normal and Abnormal Directories:

process_directories(train_dir, test_dir, output_dir)

print("Patch extraction and visualization complete.")


Patch extraction and visualization complete.


In [13]:
import os
import cv2
import numpy as np
from read_roi import read_roi_file
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

def detect_features_in_channel(channel):
    features = cv2.goodFeaturesToTrack(channel, maxCorners=200, qualityLevel=0.01, minDistance=50, blockSize=300)
    if features is not None:
        return np.int0(features)
    else:
        return np.array([])

def filter_close_points(features, min_dist=100):
    if features.ndim == 3:
        features = features.reshape(-1, 2)
    
    filtered_points = []

    for feature in features:
        if not filtered_points:
            filtered_points.append(feature)
            continue

        dists = np.sqrt(np.sum((np.array(filtered_points) - feature) ** 2, axis=1))

        if np.all(dists >= min_dist):
            filtered_points.append(feature)

    return np.array(filtered_points)

def bounding_box(roi_file_path):
    roi = read_roi_file(roi_file_path)
    for box_info in roi.values():
        if box_info['type'] == 'rectangle':
            left = box_info['left']
            top = box_info['top']
            width = box_info['width']
            height = box_info['height']
    return left, top, width, height

def visualize_and_save(image, output_path, title):
    plt.figure(figsize=(10, 5))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.savefig(output_path)
    plt.close()

def extract_and_save_patches_abnormal(folder, output_folder, visualization_folder, ideal_patch_size=(275, 300)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    if not os.path.exists(visualization_folder):
        os.makedirs(visualization_folder)
        
    for filename in os.listdir(folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(folder, filename)
            image = cv2.imread(image_path)
            visualization_image = image.copy()
            roi_path = os.path.splitext(image_path)[0] + ".roi"

            if os.path.exists(roi_path):
                blue, green, red = cv2.split(image)
                features_blue = detect_features_in_channel(blue)
                features_green = detect_features_in_channel(green)
                features_red = detect_features_in_channel(red)
                features_combined = np.vstack([f.reshape(-1, 2) for f in [features_blue, features_green, features_red] if f.size > 0])
                features_combined = np.unique(features_combined, axis=0)
                features_combined = filter_close_points(features_combined)

                if features_combined.size == 0:
                    continue  # Skip if no features found

                for point in features_combined:
                    cv2.circle(visualization_image, (int(point[0]), int(point[1])), radius=5, color=(255, 0, 0), thickness=-1)

                gt_left, gt_top, gt_width, gt_height = bounding_box(roi_path)
                cv2.rectangle(visualization_image, (gt_left, gt_top), (gt_left + gt_width, gt_top + gt_height), (0, 255, 0), 3)

                gt_center = np.array([[gt_left + gt_width / 2, gt_top + gt_height / 2]])
                distances = cdist(features_combined, gt_center, metric='euclidean')
                closest_point_index = np.argmin(distances)
                closest_point = features_combined[closest_point_index]

                sp_start_x = max(int(closest_point[0] - ideal_patch_size[1] / 2), 0)
                sp_start_y = max(int(closest_point[1] - ideal_patch_size[0] / 2), 0)
                sp_end_x = min(sp_start_x + ideal_patch_size[1], image.shape[1])
                sp_end_y = min(sp_start_y + ideal_patch_size[0], image.shape[0])

                cv2.rectangle(visualization_image, (sp_start_x, sp_start_y), (sp_end_x, sp_end_y), (0, 0, 0), 3)

                salient_patch = image[sp_start_y:sp_end_y, sp_start_x:sp_end_x]
                salient_patch_name = f"{os.path.splitext(os.path.basename(image_path))[0]}_salient_patch_{sp_start_x}_{sp_start_y}.png"
                salient_patch_path = os.path.join(output_folder, salient_patch_name)

                cv2.imwrite(salient_patch_path, salient_patch)

                # Save visualization
                visualization_path = os.path.join(visualization_folder, f"{os.path.splitext(os.path.basename(image_path))[0]}_visualization.png")
                visualize_and_save(visualization_image, visualization_path, "Salient Patch and ROI")

def extract_and_save_patches_normal(folder, output_folder, visualization_folder, ideal_patch_size=(275, 300)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    if not os.path.exists(visualization_folder):
        os.makedirs(visualization_folder)

    for filename in os.listdir(folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(folder, filename)
            image = cv2.imread(image_path)
            visualization_image = image.copy()
            h, w, _ = image.shape

            middle_x_range = (w // 4, 3 * w // 4)
            middle_y_range = (h // 4, 3 * h // 4)

            patches = []
            for _ in range(2):
                center_x = random.randint(middle_x_range[0], middle_x_range[1])
                center_y = random.randint(middle_y_range[0], middle_y_range[1])
                sp_start_x = max(int(center_x - ideal_patch_size[1] / 2), 0)
                sp_start_y = max(int(center_y - ideal_patch_size[0] / 2), 0)
                sp_end_x = min(sp_start_x + ideal_patch_size[1], w)
                sp_end_y = min(sp_start_y + ideal_patch_size[0], h)

                patch = image[sp_start_y:sp_end_y, sp_start_x:sp_end_x]
                patches.append(patch)
                patch_name = f"{os.path.splitext(os.path.basename(image_path))[0]}_normal_patch_{sp_start_x}_{sp_start_y}.png"
                patch_path = os.path.join(output_folder, patch_name)
                cv2.imwrite(patch_path, patch)

                cv2.rectangle(visualization_image, (sp_start_x, sp_start_y), (sp_end_x, sp_end_y), (0, 0, 0), 3)

            # Save visualization
            visualization_path = os.path.join(visualization_folder, f"{os.path.splitext(os.path.basename(image_path))[0]}_visualization.png")
            visualize_and_save(visualization_image, visualization_path, "Extracted Patches")

def process_directories(train_dir, test_dir, output_dir, visualization_dir, ideal_patch_size=(275, 300)):
    for data_dir in [train_dir, test_dir]:
        for class_type in ['normal', 'abnormal']:
            input_dir = os.path.join(data_dir, class_type)
            output_sub_dir = os.path.join(output_dir, os.path.basename(data_dir), class_type)
            visualization_sub_dir = os.path.join(visualization_dir, os.path.basename(data_dir), class_type)
            os.makedirs(output_sub_dir, exist_ok=True)
            os.makedirs(visualization_sub_dir, exist_ok=True)

            if class_type == 'abnormal':
                extract_and_save_patches_abnormal(input_dir, output_sub_dir, visualization_sub_dir, ideal_patch_size)
            else:
                extract_and_save_patches_normal(input_dir, output_sub_dir, visualization_sub_dir, ideal_patch_size)
    print("Patch extraction and visualization complete.")

# Define train and test directories
train_dir = 'Data/Split_Data/train'
test_dir = 'Data/Split_Data/test'
output_dir = 'Processed_Patches'
visualization_dir = 'Visualizations'

# Process the normal and abnormal directories
process_directories(train_dir, test_dir, output_dir, visualization_dir)

print("Patch extraction and visualization complete.")


Patch extraction and visualization complete.
